# This file is for functional strategies only!

In [8]:
def add_trend_signal(df):
    df['Trend'] = 0
    for i in range(5, len(df)):
        window = df.iloc[i-5:i]
        if all(window['Close'] > window['EMA_200']):
            df.loc[i, 'Trend'] = 1
        elif all(window['Close'] < window['EMA_200']):
            df.loc[i, 'Trend'] = -1

    return df

In [11]:
def add_total_signal(df):
    # Strategy: Trade with trend, rsi > 50, and engulfing candle
    # Trade Management: Stop loss @ 2x candle width, 1.5 risk to reward ratio
    # -------------------------------------------------------------------------------
    # Trial 1: Very negative returns, going to try just buys
    # Trial 2: Only did buy positions, positive returns, not as high as buy & hold returns
    # Results: ~35% win rate with positive decent returns (14.5%) for 2.0 ratio
    
    # df['engulfing_signal'] = 0
    # df.loc[ (df['Close'] > df['EMA_200']) & (df['RSI'] > 50) & (df['Engulfing'] == 100), 'engulfing_signal'] = 1

    # Strategy: Trade with trend, if candle closes outside of bollinger bands, make a limit order at 3% below closing price
    # Trade Management: Limit order is valid for 5 days, stop loss @ 3% below limit, 1.5 risk to reward ratio
    # -------------------------------------------------------------------------------
    # Trial 1: ~40% win rate with positive decent returns (11.5%) for 1.9 ratio
    # Trail 2: Sell when RSI crosses 50 line, much better returns (~27%)
    # Trial 3: Same trade management, only buy signals. ~45% return rate and 72% win rate
    df['bb_signal'] = 0
    df.loc[ (df['Trend'] == 1) & (df['Close'] < df['BBL_20_2.0']), 'bb_signal'] = 1
    # df.loc[ (df['Trend'] == -1) & (df['Close'] > df['BBU_20_2.0']), 'bb_signal'] = -1

    # Strategy: Trade with trend, look for MACD crossover above or below the zero line
    # Trade Management: Optimize parameters for sl and tp
    # -------------------------------------------------------------------------------
    # Trial 1: ~ 60% win rate with ~16% return rate, begin paper trade testing
    # df['macd_signal'] = 0
    # df.loc[ (df['Trend'] == 1) & (df['MACD_cross'] == 1) & (df['MACD_12_26_9'] < 0), 'macd_signal'] = 1

    #Strategy: Trade with trend, buy if 50 SMA crosses above 200 SMA, sell is 50 SMA crosses below 200 SMA
    #Trade Management: Trailing stop loss @ 2x ATR
    # -------------------------------------------------------------------------------
    # Trial 1:

    return df

In [12]:
import yfinance as yf
from yahoo_fin import stock_info as si
import pandas_ta as ta
import numpy as np
import pandas as pd

def import_data(stock, startDate):
    data = yf.download(stock, start=startDate, interval='1d')

    data['EMA_200'] = ta.ema(data['Close'], length=200)
    data['SMA_50'] = ta.sma(data['Close'], length=50)
    data['SMA_200'] = ta.sma(data['Close'], length=200)
    data['RSI'] = ta.rsi(data['Close'], length=14)
    data['ATR'] = ta.atr(data['High'], data['Low'], data['Close'], length=14)
    data['Engulfing'] = data.ta.cdl_pattern(name="engulfing")
    bbands = ta.bbands(data['Close'], length=20, std=2)
    macd = ta.macd(data['Close'])
    data = data.join(bbands)
    data = data.join(macd)
    
    data.reset_index(inplace=True)
    
    data = add_trend_signal(data)
    # data = add_macd_signal(data)
    data = add_total_signal(data)
    # data = add_sma_signal(data)

    return data

In [13]:
def calc_bb(ticker, risk):
    df = import_data(ticker, '2022-01-01')
    today = df.iloc[-1]
    curr = today.Close
    order = curr - (curr * .03)
    stop = order - (order * .03)
    shares = risk / (order - stop)

    order = round(order, 2)
    shares = round(shares, 2)

    print(f'Buy {ticker} at {order}, Shares: {shares}')

In [14]:
def find_buys():
    bb_buys = []
    engulfing_buys = []
    macd_buys = []
    sp500_url = 'https://en.wikipedia.org/wiki/List_of_S%26P_500_companies'
    sp500_table = pd.read_html(sp500_url)
    TICKERS = sp500_table[0]['Symbol'].tolist()

    for ticker in TICKERS:
        try:
            df = import_data(ticker, '2023-01-01')
            # engulfing_signal = df.iloc[-1]['engulfing_signal']
            bb_signal = df.iloc[-1]['bb_signal']
            # macd_signal = df.iloc[-1]['macd_signal']
            # if engulfing_signal == 1:
            #     engulfing_buys.append(ticker)
            if bb_signal != 0:
                bb_buys.append(ticker)
            # if macd_signal != 0:
            #     macd_buys.append(ticker)
        except Exception as e:
            print(ticker, e)

    print(f'BB Buy Signals: {bb_buys}')
    for ticker in bb_buys:
        calc_bb(ticker, 20)
        calc_bb(ticker, 1000)
    # print(f'Engulfing Buy Signals: {engulfing_buys}')
    # for ticker in engulfing_buys:
    #     calc_engulfing(ticker)
    # print(f'MACD Signals: {macd_buys}')
    # for ticker in macd_buys:
    #     calc_macd(ticker)

In [15]:
find_buys()

[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%*******

AMTM 'NoneType' object is not iterable


[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%*******

BRK.B 'NoneType' object is not iterable



[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed

1 Failed download:
['BF.B']: YFP

$BF.B: possibly delisted; No price data found  (1d 2023-01-01 -> 2024-10-01)
BF.B 'NoneType' object is not iterable



[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%******

KeyboardInterrupt: 